In [ ]:
%reload_ext autoreload
%autoreload 2


import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
import pickle
from tqdm import tqdm
import scipy

import flexiznam as flz
from cottage_analysis.preprocessing import synchronisation
from cottage_analysis.analysis import spheres, find_depth_neurons, common_utils, fit_gaussian_blob, size_control
from cottage_analysis.plotting import basic_vis_plots, plotting_utils
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, close_loop_rsof, get_session_list, open_loop_rsof
from v1_depth_analysis.v1_manuscript_2023 import common_utils as v1_common_utils

In [ ]:
VERSION = 3
SAVE_ROOT = '/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver'+str(VERSION)+'/fig4/'
os.makedirs(SAVE_ROOT, exist_ok=True)

In [ ]:
# Load example session
project='hey2_3d-vision_foodres_20220101'
session_name='PZAH6.4b_S20220524'
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={'anatomical_only':3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=2,
    return_volumes=True,
)

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name, flexilims_session=flexilims_session, project=None, conflicts="skip"
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)

In [ ]:
# Concatenate all sessions
project='hey2_3d-vision_foodres_20220101'
flexilims_session = flz.get_flexilims_session(project)
results_all = v1_common_utils.concatenate_all_neurons_df(flexilims_session=flexilims_session,
                                                         session_list=get_session_list.get_all_sessions(project=project,mouse_list=[
                                                                                                                                    "PZAH6.4b",
                                                                                                                                    "PZAG3.4f",
                                                                                                                                    "PZAH8.2h", 
                                                                                                                                    "PZAH8.2i", 
                                                                                                                                    "PZAH8.2f", 
                                                                                                                                    "PZAH10.2d", 
                                                                                                                                    "PZAH10.2f"
                                                                                                                                    ], 
                                                                                                        closedloop_only=False, 
                                                                                                        openloop_only=True),
                                                         filename="neurons_df.pickle",
                                                         cols=['roi','best_depth','preferred_depth_closedloop','preferred_depth_closedloop_crossval',
                                                               'depth_tuning_test_rsq_closedloop','depth_tuning_test_spearmanr_pval_closedloop',
                                                               'preferred_RS_closedloop_g2d','preferred_RS_closedloop_crossval_g2d',
                                                               'preferred_OF_closedloop_g2d','preferred_OF_closedloop_crossval_g2d',
                                                               'rsof_rsq_closedloop_crossval_g2d','rsof_rsq_closedloop_g2d',
                                                               'preferred_RS_openloop_actual_g2d','preferred_OF_openloop_actual_g2d',
                                                               'rsof_rsq_openloop_actual_g2d',],
                                                         
                                                         read_iscell=True,
                                                         verbose=True)
results_all["preferred_RS_closedloop_g2d"] = results_all["preferred_RS_closedloop_g2d"]*100
results_all["preferred_depth_closedloop"] = results_all["preferred_depth_closedloop"]*100
results_all["preferred_OF_closedloop_g2d"] = np.degrees(results_all["preferred_OF_closedloop_g2d"])
results_all["preferred_RS_openloop_actual_g2d"] = results_all["preferred_RS_openloop_actual_g2d"]*100
results_all["preferred_OF_openloop_actual_g2d"] = np.degrees(results_all["preferred_OF_openloop_actual_g2d"])


In [ ]:
fig=plt.figure()
fontsize_dict = {"title": 5, "label": 7, "tick": 5, "legend": 5}
open_loop_rsof.plot_speeds_scatter(
    fig=fig,
    neurons_df=results_all,
    xcol="preferred_RS_closedloop_g2d",
    ycol="preferred_RS_openloop_actual_g2d",
    xlabel="Running speed closedloop(cm/s)",
    ylabel="Running speed openloop (cm/s)",
    s=5,
    alpha=0.2,
    c='g',
    plot_x=0.3,
    plot_y=0,
    plot_width=0.3,
    plot_height=0.3,
    cbar_width=0.01,
    plot_diagonal=True,
    fontsize_dict=fontsize_dict,)

open_loop_rsof.plot_speeds_scatter(
    fig=fig,
    neurons_df=results_all,
    xcol="preferred_OF_closedloop_g2d",
    ycol="preferred_OF_openloop_actual_g2d",
    xlabel="Optic flow speed closed loop" +u'(\N{DEGREE SIGN}/s)',
    ylabel="Optic flow speed open loop "+u'(\N{DEGREE SIGN}/s)',
    s=5,
    alpha=0.2,
    c='b',
    plot_x=0.7,
    plot_y=0,
    plot_width=0.3,
    plot_height=0.3,
    cbar_width=0.01,
    plot_diagonal=True,
    fontsize_dict=fontsize_dict,)

# decoder plots

# example open loop matrix, corner of RS 0 and OF 0
EXAMPLE_ROIS = [121, 387, 440]
#312, 424, 477, 121, 446, 441, 324, 405
# 387, 357
# 293, 515, 440, 164
for iroi, roi in enumerate(EXAMPLE_ROIS):
    if iroi != (len(EXAMPLE_ROIS)-1):
        xlabel=""
    else:
        xlabel="Running speed (cm/s)"
    if iroi != (len(EXAMPLE_ROIS)//2):
        ylabel=""
    else:
        ylabel="Optical flow speed "+u'(\N{DEGREE SIGN}/s)'
    
    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=3,
        linecolor="k",
        fit_linecolor="r",
        closed_loop=1,
        plot_x=0.3,
        plot_y=1-0.28*iroi,
        plot_width=0.2,
        plot_height=0.2,
        fontsize_dict=fontsize_dict,
        )
    if iroi != len(EXAMPLE_ROIS)-1:
        plt.xlabel('')
    
    # close_loop_rsof.plot_speed_tuning(
    #     fig=fig,
    #     neurons_df=neurons_df_example,
    #     trials_df=trials_df_example,
    #     roi=roi,
    #     is_closed_loop=1,
    #     nbins=10,
    #     which_speed="RS",
    #     speed_min=0.01,
    #     speed_max=1.5,
    #     speed_thr=0.01,
    #     smoothing_sd = 1,
    #     markersize=4,
    #     linewidth=2,
    #     plot_x=0.2,
    #     plot_y=1-0.25*iroi,
    #     plot_width=0.15,
    #     plot_height=0.15,
    #     fontsize_dict=fontsize_dict,
    #     legend_on=False,
    # )
    # if iroi != len(EXAMPLE_ROIS)-1:
    #     plt.xlabel('')
    # plt.ylabel('')
    
    # close_loop_rsof.plot_speed_tuning(
    #     fig=fig,
    #     neurons_df=neurons_df_example,
    #     trials_df=trials_df_example,
    #     roi=roi,
    #     is_closed_loop=1,
    #     nbins=10,
    #     which_speed="OF",
    #     speed_min=0.01,
    #     speed_max=1.5,
    #     speed_thr=0.01,
    #     smoothing_sd = 1,
    #     markersize=4,
    #     linewidth=2,
    #     plot_x=0.4,
    #     plot_y=1-0.25*iroi,
    #     plot_width=0.15,
    #     plot_height=0.15,
    #     fontsize_dict=fontsize_dict,
    #     legend_on=True,
    # )
    # if iroi != len(EXAMPLE_ROIS)-1:
    #     plt.xlabel('')
    # plt.ylabel('')
    
    # if iroi != len(EXAMPLE_ROIS)-1:
    #     xlabel=''
    # else:
    #     xlabel='Running speed (cm/s)'
    
    matrix = close_loop_rsof.plot_RS_OF_matrix(fig=fig,
                                        trials_df=trials_df_example,
                                        roi=roi,
                                        log_range={
                                            "rs_bin_log_min": 0,
                                            "rs_bin_log_max": 2.5,
                                            "rs_bin_num": 6,
                                            "of_bin_log_min": -1.5,
                                            "of_bin_log_max": 3.5,
                                            "of_bin_num": 11,
                                            "log_base": 10,
                                        },
                                        is_closed_loop = 1,
                                        vmin=None,
                                        vmax=None,
                                        xlabel=xlabel,
                                        ylabel=ylabel,
                                        plot_x=0.6,
                                        plot_y=1-0.28*iroi,
                                        plot_width=0.2,
                                        plot_height=0.2,
                                        cbar_width=0.005,
                                        fontsize_dict=fontsize_dict,
    )
    
        
    _ = close_loop_rsof.plot_RS_OF_matrix(fig=fig,
                                    trials_df=trials_df_example,
                                    roi=roi,
                                    log_range={
                                        "rs_bin_log_min": 0,
                                        "rs_bin_log_max": 2.5,
                                        "rs_bin_num": 6,
                                        "of_bin_log_min": -1.5,
                                        "of_bin_log_max": 3.5,
                                        "of_bin_num": 11,
                                        "log_base": 10,
                                    },
                                    is_closed_loop = 0,
                                    vmin = np.nanmax([0, np.percentile(matrix[1:, 1:].flatten(), 1)]),
                                    vmax= np.nanmax(matrix[1:, 1:].flatten()),
                                    xlabel=xlabel,
                                    ylabel='',
                                    plot_x=0.85,
                                    plot_y=1-0.28*iroi,
                                    plot_width=0.2,
                                    plot_height=0.2,
                                    cbar_width=0.005,
                                    fontsize_dict=fontsize_dict,
    )

plt.savefig(SAVE_ROOT+'fig4.pdf', bbox_inches='tight', transparent=True)
plt.savefig(SAVE_ROOT+'fig4.png', bbox_inches='tight', transparent=True, dpi=300)

In [ ]:

# Minor:
# - add an illustration of closed loop & open loop
# - DONE: get rid of the speed plots?
# - DONE: separate out the two extra columns of the matrix
# - DONE: add dotted line for scatter plot
# - change spacing of matrix to 2